<a href="https://colab.research.google.com/github/Christopher-Ramirez-S/An-lisis-de-datos/blob/main/lab_03_final_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Adrian Rodriguez
#Rafael Garcia
#Cristopher Ramirez
#Eldon Calwell Salazar
#Aaron Navarro Balmaceda

import requests
import json
import concurrent.futures
import threading
import time
import logging
import os
from tqdm import tqdm
import datetime
import pandas as pd
import csv
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns


# Try to read the file, handling potential errors
try:
    df = pd.read_csv('/content/TMDB_movie_dataset_v11.csv')
    data = pd.DataFrame(df)
except pd.errors.ParserError as e:
    print(f"An error occurred while reading the CSV file: {e}")
    # You might want to add more specific error handling or logging here

In [25]:
#datetime y coerce son funciones de pandas para convertir correctamente la fecha y si no tiene un formato lo remplaza con "not a date time"
data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')


In [31]:
start_date = pd.to_datetime('1910-01-01')
end_date = pd.to_datetime('2024-07-31')

filtered_data = data[(data['release_date'] >= start_date) & (data['release_date'] <= end_date)]
filtered_data

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083004,706809,Bon Jovi: Live In Japan,0.000,0,Released,1988-12-31,0,0,False,NaN,...,Bon Jovi: Live In Japan,Recorded live at Tokyo Dome in Japan on 31 Dec...,0.600,/tUOzC6YDI0hWa2izlK3DmsBLLC6.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1083006,706811,Slow News,0.000,0,Released,2020-05-19,0,93,False,NaN,...,Slow News,"“Slow News” is a contemporary story, a look on...",0.600,/bMvp4D2yz9NxRjzlPK6X1cIV9lf.jpg,NaN,Documentary,IK Produzioni,Italy,"English, Italian",NaN
1083007,706812,Levine,0.000,0,Released,2017-06-10,0,102,False,NaN,...,Levine,"Plagued by Survivors Syndrome, Madeleine McNam...",0.600,/jOego3urOfEZ0oyvYvTjF3TvyUX.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1083008,706814,The Doodlers,0.000,0,Released,1975-01-01,0,5,False,NaN,...,The Doodlers,Miss Nose and her class engage in drawings in ...,0.600,NaN,NaN,Animation,NaN,NaN,NaN,NaN


In [32]:
#eldon querys
#pelicula más vieja
#dropna es un metodo para remover columnas con values faltantes en el dataframe, pero solo para la comuna "release date" en este caso
data_con_fechas = filtered_data.dropna(subset=['release_date'])

#filtra el dato del minimo con min de "release date más viejo"
pelicula_vieja = data_con_fechas.loc[data_con_fechas['release_date'].idxmin()]

print("Película más vieja:")
print("Título:", pelicula_vieja['title'])
print("Fecha de lanzamiento:", pelicula_vieja['release_date'])

Película más vieja:
Título: The Doctor's Secret
Fecha de lanzamiento: 1910-01-01 00:00:00


In [34]:
#Cris Peliculas con mayor popularidad desde 1910 a 2024
filtered_data.sort_values(by=['popularity'], ascending=False)[['popularity','release_date','revenue','title']]. head(5)

,popularity,release_date,revenue,title
3868,2994.357,2023-08-16,124818235,Blue Beetle
5052,2680.593,2023-08-09,114800000,Gran Turismo
50447,2020.286,2020-01-16,0,A Female Boss with Big Tits and Her Cherry Boy...
7919,1692.778,2023-09-06,231200000,The Nun II
2129,1567.273,2023-08-02,384056482,Meg 2: The Trench


In [44]:
#aron
#busca las producciones que hubo en CR en 2017
costa_rica_2017_movies = data[(data['production_countries'] == "Costa Rica") & (data['release_date'].dt.year == 2017)][['title', 'revenue', 'budget']].head(5)
print(costa_rica_2017_movies)

                             title  revenue  budget
38975   Keylor Navas, Hombre de Fe        0       0
80118    Buscando a Marcos Ramírez        0       0
130712     The Heat After the Rain        0       0
164985                Amor Viajero        0       0
300562        Casa en Tierra Ajena        0       0


In [45]:
##Adrian query
#Sacar las 10 peliculas con mayor votacion en todos los años

MayorVot=filtered_data.sort_values(by='vote_count',ascending=False)
MayorVot.head(10)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
5,293660,Deadpool,7.606,28894,Released,2016-02-09,783100000,108,False,/en971MEXui9diirXlogOrPKmsEn.jpg,...,Deadpool,The origin story of former Special Forces oper...,72.735,/zq8Cl3PNIDGU3iWNRoc5nEZ6pCe.jpg,Witness the beginning of a happy ending.,"Action, Adventure, Comedy","20th Century Fox, The Donners' Company, Genre ...",United States of America,English,"superhero, anti hero, mercenary, based on comi..."
6,299536,Avengers: Infinity War,8.255,27713,Released,2018-04-25,2052415039,149,False,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,...,Avengers: Infinity War,As the Avengers and their allies have continue...,154.340,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,An entire universe. Once and for all.,"Adventure, Action, Science Fiction",Marvel Studios,United States of America,"English, Xhosa","sacrifice, magic, superhero, based on comic, s..."
7,550,Fight Club,8.438,27238,Released,1999-10-15,100853753,139,False,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,...,Fight Club,A ticking-time-bomb insomniac and a slippery s...,69.498,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,Mischief. Mayhem. Soap.,Drama,"Regency Enterprises, Fox 2000 Pictures, Taurus...",United States of America,English,"dual identity, rage and hate, based on novel o..."
8,118340,Guardians of the Galaxy,7.906,26638,Released,2014-07-30,772776600,121,False,/uLtVbjvS1O7gXL8lUOwsFOH4man.jpg,...,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",33.255,/r7vmZjiyZw9rpJMQJdXpjgiCOk9.jpg,All heroes start somewhere.,"Action, Science Fiction, Adventure",Marvel Studios,United States of America,English,"spacec

In [46]:
#Adrian query
#Suma la cantidad de votos que a recibido todas las peliculas de una productora, las 10 primeras de mayor a menor

VotosPorProd = filtered_data.groupby('production_companies')['vote_count'].sum()
VotosPorProd.sort_values(ascending=False).head(10)

,vote_count
production_companies,
Marvel Studios,433824
"Pixar, Walt Disney Pictures",169137
Paramount,115439
Walt Disney Productions,112438
Pixar,110702
Lucasfilm Ltd.,100019
"Warner Bros. Pictures, Heyday Films",87288
"Walt Disney Pictures, Walt Disney Feature Animation",79750
Universal Pictures,77460


In [47]:
#Adrian query
# Filtra todas las peliculas que esten en idioma Ingles
PeliIngles = filtered_data[filtered_data['spoken_languages'].str.contains('En', na=False)]
PeliIngles

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082946,706680,Lady O'Loughlin,0.000,0,Released,2020-05-04,0,53,False,NaN,...,Lady O'Loughlin,A documentary about a recovering alcoholic and...,0.604,NaN,An in-depth look at comedian Fiona O'Loughlin'...,Documentary,The Producer Boy,Australia,English,"stand-up comedy, alcoholic"
1082958,706710,Am I Trustworthy?,0.000,0,Released,1950-12-31,0,11,False,NaN,...,Am I Trustworthy?,Eddie Johnson is disappointed that none of his...,0.600,/bcgmdpvrxOlUgso2ev6k8k0YC7h.jpg,NaN,NaN,NaN,NaN,English,NaN
1082986,706874,Sensitive and in Love,0.000,0,Released,2020-01-29,0,122,False,/1GAtrwma6xg3dTHzv4BczRi93XW.jpg,...,Sensitive and in Love,A millennial writer from San Francisco travels...,1.492,/266GkBOId29XeKFyRQMUUgawsZZ.jpg,NaN,Drama,The Global Touch Group,United States of America,English,NaN
1083001,706806,Pleasuring Her Pussy,0.000,0,Released,2017-01-01,0,123,True,/9T8sVRpk7sTFPVRYJUSBfv0r91a.jpg,...,Pleasuring Her Pussy,If there is not a single cock around and bored...,0.600,/a10cpO8rUyFhDi3CMrjyo9PIPOg.jpg,NaN,NaN,DDF,NaN,English,NaN
